# Generating topics for venue tips in London
* prepare tips data
* preprocessing tips
* generating topics
* evaluation

### 1.prepare tips data
We collect tips from Foursquare. 

There are around 11000 places and about 1700 places have more 200 tips.
All of the tips(English tips) were collected previously. 

In [ ]:
# import useful libs
import graphlab as gl
import pandas as pd
import json
import re
import pyLDAvis 
import pyLDAvis.graphlab

In [65]:
# load the pre-processed data
sf = gl.SFrame.read_json('london_venue.json')
#sf.head()

Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\Recommender System\london_venue.json

Parsing completed. Parsed 1 lines in 0.34851 secs.

### 2. preprocessing
* put all tips for one venue together, store them in a new column 'all_tips'

In [53]:
# put all tips together
def get_feature(row):
    feature=''
    if row['tips']!=None:
        for t in row['tips']:
            feature += t
    
    tags = 5*row['tags']
    for tag in tags:
        feature += tag+' '
    if row['description']!=None:
        feature += row['description']
        feature += ' '
        feature += row['description']
    return feature
sf['feature'] = sf.apply(lambda x: get_feature(x))

   * Text cleaning
   * Lemmatization

In [54]:
# get words, dropping punctuations etc.
sf['feature'] = sf['feature'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
docs = gl.text_analytics.tokenize(sf['feature'])

In [56]:
# lemmatization
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
docs_lemmatized = docs.apply(lambda x: [wordnet_lemmatizer.lemmatize(w) for w in x])
#docs_lemmatized[0]

* Tokenization
* Bag-of-words representation
* Stop words and less frequent words removal

In [57]:
# Bag-of-words
docs_cleaned = gl.text_analytics.count_words(docs_lemmatized)
# Remove stop words
docs_cleaned = docs_cleaned.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)
# Remove less freq words
docs_cleaned = docs_cleaned.dict_trim_by_values(2)

###  3. Gnerate topics
* remove docs which has less than 3 keywords
* create a model
* check and evaluate

In [58]:
# remove docs which has less than 3 keywords
ix = docs_cleaned.apply(lambda x:len(x.keys())>3)
docs_new = docs_cleaned[ix]

In [59]:
# Show how many docs have been removed
print 1.0*len(docs_new)/len(docs_cleaned)

0.407899131597


In [63]:
# create a topic model 
#associations = gl.SFrame({'word':['local','authentic', 'london', 'good', 'lunch','delicious','amazing','lover','walk'],
#                                'topic': [0, 0, 0, 1, 1,2,2,3,3]})

topic_model = gl.topic_model.create(docs_new,num_topics=20, num_iterations=200)

Learning a topic model

Number of documents      4885

Vocabulary size     11203

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 170       | 4.29s         | 4.54637e+006   | 0               |

| 180       | 4.53s         | 9.09273e+006   | 0               |

| 190       | 4.78s         | 4.54491e+006   | 0               |

| 200       | 5.06s         | 4546656        | 0               |

+-----------+---------------+----------------+-----------------+

In [ ]:
for i in range(50):
    print 'topic ',i,topic_model.get_topics(num_words=6,output_type='topic_words')[i]['words']

In [ ]:
# These are the 5th topic words
print topic_model.get_topics(num_words=10,output_type='topic_words')[11]

In [ ]:
docs_in_topic_5 = docs_new[topic_model.predict(docs_new)==11]

In [ ]:
# select venues which are predicted to be topic 5
sf_new = sf[ix]
venue_in_topic5 = sf_new[topic_model.predict(docs_new)==11]

In [ ]:
venue_in_topic5['category','name','tips'][21]

In [64]:
#Automatic Rendering of Visualisation
pyLDAvis.enable_notebook()
pyLDAvis.graphlab.prepare(topic_model, docs_new)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
9      7.921754        1       1 -0.102102 -0.092930
2      7.274937        1       2  0.097458 -0.159244
4      6.099108        1       3 -0.153346 -0.024846
8      6.046761        1       4  0.230335  0.162160
19     5.408817        1       5  0.239300  0.028993
17     5.137246        1       6 -0.086811 -0.136008
0      5.061063        1       7  0.026953 -0.160347
3      5.026168        1       8  0.124909  0.079500
1      5.009270        1       9  0.129273 -0.066283
15     4.965392        1      10  0.183430  0.141222
12     4.905313        1      11 -0.234029  0.030829
18     4.873809        1      12 -0.104854 -0.065003
10     4.721290        1      13 -0.163131  0.108284
14     4.648628        1      14  0.031527 -0.068578
6      4.632128        1      15 -0.073627 -0.066793
16     4.432482        1      16  0.132611 -0.110041
5      3.870311        1      17 -0.029753 -0.110239
11     3.709865        1      18  0.179533  0.143129
7      3.373011        1      19 -0.228761  0.102306
13     2.882648        1      20 -0.198916  0.263886, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
4378   Default  14278.000000           food  14278.000000  30.0000  30.0000
2993   Default   5546.000000         coffee   5546.000000  29.0000  29.0000
7237   Default  13808.000000           good  13808.000000  28.0000  28.0000
8588   Default   9784.000000         london   9784.000000  27.0000  27.0000
271    Default   7263.000000        service   7263.000000  26.0000  26.0000
2146   Default  16604.000000          great  16604.000000  25.0000  25.0000
10906  Default   3193.000000           view   3193.000000  24.0000  24.0000
4256   Default   2926.000000            pub   2926.000000  23.0000  23.0000
10141  Default   2762.000000          hotel   2762.000000  22.0000  22.0000
7347   Default   3071.000000           room   3071.000000  21.0000  21.0000
9381   Default  10226.000000          place  10226.000000  20.0000  20.0000
8865   Default   2787.000000           beer   2787.000000  19.0000  19.0000
5912   Default   4223.000000            bar   4223.000000  18.0000  18.0000
7086   Default   2674.000000         burger   2674.000000  17.0000  17.0000
3202   Default   7606.000000             wa   7606.000000  16.0000  16.0000
6506   Default   2006.000000          pizza   2006.000000  15.0000  15.0000
8071   Default   4766.000000          staff   4766.000000  14.0000  14.0000
6899   Default   6587.000000           nice   6587.000000  13.0000  13.0000
10990  Default   3967.000000     restaurant   3967.000000  12.0000  12.0000
10771  Default   2344.000000       cocktail   2344.000000  11.0000  11.0000
164    Default   2050.000000           wine   2050.000000  10.0000  10.0000
9901   Default   2066.000000            tea   2066.000000   9.0000   9.0000
5697   Default   5043.000000        amazing   5043.000000   8.0000   8.0000
3836   Default   2435.000000      breakfast   2435.000000   7.0000   7.0000
98     Default   1726.000000          music   1726.000000   6.0000   6.0000
7331   Default   1845.000000           park   1845.000000   5.0000   5.0000
8661   Default   2898.000000           free   2898.000000   4.0000   4.0000
10874  Default   1918.000000        chicken   1918.000000   3.0000   3.0000
2594   Default   2331.000000          drink   2331.000000   2.0000   2.0000
8267   Default   1232.000000        italian   1232.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
7190   Topic20     87.264617        creates     89.096957   3.5257  -5.2312
9753   Topic20     85.448496  communication     87.280836   3.5252  -5.2522
987    Topic20     83.632375   preservative     85.464715   3.5248  -5.2737
6388   Topic20     83.632375       avoiding     85.4647